In [1]:
from openai import OpenAI

import json
with open("history_5.jsonl", 'r', encoding='utf-8') as f:
    datas = [json.loads(line) for line in f]

In [2]:
user_utterances = []
for data in datas:
    for key, value in data.items():
        if key.startswith("turn_") and isinstance(value, dict):
            user_utterances.append(value.get("user", ""))

In [3]:
len(user_utterances)

1988

In [4]:
class LLM_call:
    def __init__(self, llm_model=None):
        self.llm_model = llm_model

    def call_llm(self, llm_input):
        raise NotImplementedError

    def __call__(self, llm_input):
        return self.call_llm(llm_input)
class OPENAI_call(LLM_call):
    def __init__(self, llm_model=None):
        super(OPENAI_call, self).__init__(llm_model)
        self.model = llm_model
        self.temperature = 0
        self.max_tokens = 512
        self.seed = 39
        self.client = OpenAI()


    # @retry(stop_max_attempt_number=10)
    def call_llm(self, llm_input):
        # try:
        # print("self.model:", self.model)
        output = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": llm_input}
            ],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            seed=self.seed
        )
        # print(output)

        return output.choices[0].message.content
        # except Exception as e:
        #     LOGGER.warning(f"Failed to call LLM model: {e}")
        #     return None

    def set_temperature(self, temperature):
        self.temperature = temperature

    def set_max_tokens(self, max_tokens):
        self.max_tokens = max_tokens

    def set_seed(self, seed: int):
        self.seed = seed

In [5]:
import os
# 设置环境变量（在程序运行时生效，但不会写入系统）
os.environ["OPENAI_API_KEY"] = "sk-G6XflAR04SvxaTl1QhmIl6HimueZ0ZPYFeEVD78gELMRqth5"
os.environ["OPENAI_BASE_URL"] = "https://api.chatanywhere.tech"

In [6]:
prompt = """你是一位对话推荐系统领域的专家。

你的任务是：根据用户的一段对话内容，判断用户在对话中使用了哪一种或哪几种推荐交互策略，并说明判断理由。

可选策略包括以下3类（每条策略后都附有使用时的典型场景说明）：


1. 基于推荐结果给出反馈（Feedback on Recommendation）：
   用户对系统已经给出的推荐内容表达正向或负向情感态度。
   示例：这部电影听起来不错，我喜欢爱情片。

2. 寻求推荐（Seek Recommendation）：
   用户主动请求系统进行推荐，通常是对话的起始行为。
   示例：你能推荐一些适合聚餐的餐馆吗？

3. 属性询问（Attribute Inquiry）：
   用户对已经推荐的项目表示出一定兴趣，但想进一步了解其某些属性（如导演、位置、评分等）。
   示例：这家餐厅在哪儿？营业时间是几点？

用户的对话为：{user_utt}
【输出要求】  
请你严格按照以下 JSON 格式输出内容：
{{
  "判断理由": "<请在此填写你的分析过程和判断依据>",
  "策略": ["<填写一个或多个策略名称，例如：寻求推荐>"]
}} 
每个策略之间用英文的逗号隔开。
"""

In [7]:
LLM_config_action = {
    "model": "gpt-4o-mini",
    "temperature": 0,
    "max_tokens": 9000,
    "seed": 39,
}

Judge = OPENAI_call(LLM_config_action["model"])
Judge.set_temperature(LLM_config_action["temperature"])
Judge.set_max_tokens(LLM_config_action["max_tokens"])
Judge.set_seed(LLM_config_action["seed"])

In [8]:
user_utterances[0]

'请推荐一些适合独自用餐的美食。  '

In [9]:
print(Judge(prompt.format(user_utt="能不能给我推荐一些附近的米粉面馆呢？")))

{
  "判断理由": "用户在对话中主动请求系统提供推荐，表明他们希望获得一些关于附近米粉面馆的建议。这种行为符合‘寻求推荐’策略的定义，通常为对话的起始行为。",
  "策略": ["寻求推荐"]
}


In [10]:
from concurrent.futures import ThreadPoolExecutor, as_completed

dict = {
    "寻求推荐": 0,
    "基于推荐结果给出反馈": 0,
    "偏好澄清": 0,
    "属性询问": 0
}

def process_utterance(utt):
    res = Judge(prompt.format(user_utt=utt))
    res = eval(res)
    return res

user_utterances = user_utterances  # 保证在有效范围内

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_utterance, utt) for utt in user_utterances]
    
    for future in as_completed(futures):
        try:
            res = future.result()
            print(res)
            actions = res['策略']
            for action in actions:
                dict[action] += 1
        except Exception as e:
            print(f"Error processing utterance: {e}")

print(dict)

{'判断理由': '用户在对话中主动询问系统是否可以推荐一些意面或披萨的地方，这表明用户希望获得推荐信息，因此属于寻求推荐策略。', '策略': ['寻求推荐']}
{'判断理由': '用户主动请求系统进行推荐，明确表示希望获取适合独自用餐的美食。这表明用户在寻求推荐，与策略定义相符。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中主动询问系统是否可以推荐江汉路附近的餐厅，这表明用户希望系统提供相关的餐饮推荐，因此使用了寻求推荐的策略。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中提到他们的位置（三里屯）和他们的需求（米其林或牛排馆），显然是在主动请求系统推荐合适的餐馆。这符合寻求推荐的策略。', '策略': ['寻求推荐']}
{'判断理由': '用户表示自己升职了并想和朋友庆祝，随后直接请求推荐高档美食，明显是在主动寻找推荐，因此符合寻求推荐的策略。没有给出现有推荐的反馈或询问具体属性，因此不适用其他策略。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中明确表示出对食物的需求，并主动询问系统是否能做出推荐，这表明用户在寻求推荐。对话的整体内容是用户请求推荐内容的起始行为。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中询问是否有推荐的韩餐馆，这表明他们正在主动请求系统提供推荐。这种行为符合寻求推荐的策略，因为用户希望得到具体的建议。', '策略': ['寻求推荐']}
{'判断理由': "用户在对话中明确表示自己在南京路，并且心情不佳，想要通过美味的浙菜来改善情绪。这表明用户主动请求系统进行推荐，符合寻求推荐的策略。用户使用了'请问您能推荐一些餐馆吗？'这一句，直接请求推荐，进一步确认了其寻求推荐的意图。", '策略': ['寻求推荐']}
{'判断理由': '用户主动请求系统进行推荐，特别是在对餐饮类型（湘菜或饺子混沌）和特定健康需求（低盐）提出了具体的要求，这表明用户在积极寻求推荐。这符合‘寻求推荐’策略的定义。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中询问具体地址，说明他们已经对推荐的餐厅表示出一定的兴趣，并希望了解更多细节。这属于对已经推荐的项目的属性询问。用户并没有主动请求推荐，也没有对已推荐的内容反馈情绪，只是进一步了解相关信息，因此

In [11]:
len(user_utterances)

1988